In [1]:
# we load the csv into pandas
import pandas as pd
import numpy as np

date_df = pd.read_csv('dataset_data.csv', low_memory=False)
id_df = pd.read_csv('dataset_id.csv', low_memory=False)

In [ ]:
# We drop the index



In [2]:
def clean_duplicates(df):
    # Check for duplicates based on the 'id' column
    duplicate_ids = df[df.duplicated(subset='id', keep=False)]

    # Group by 'id' to check if rows are identical
    for id_value, group in duplicate_ids.groupby('id'):
        if group.duplicated(keep=False).all():
            # All rows with this ID are identical; keep the first and drop the rest
            df = df.drop_duplicates(subset=['id'], keep='first')
        else:
            # Rows with this ID differ; print them for manual inspection
            print(f"Conflicting rows for ID {id_value}:\n{group}")

In [4]:
clean_duplicates(date_df)

Conflicting rows for ID 3124607:
              data                                          entradeta  \
33218   2021-11-04  S'hauran tornat, un any abans del previst, tot...   
144914  2021-11-04  S'hauran tornat, un any abans del previst, tot...   

                                               permatitle avantitol  \
33218   Els treballadors públics rebran abans de final...       NaN   
144914  Els treballadors públics rebran abans de final...       NaN   

                                               tematiques  \
33218   [{'id': 'PUACT_ECONOM', 'desc': 'Economia (Act...   
144914  [{'id': 'PUACT_ECONOM', 'desc': 'Economia (Act...   

                                                     tags       id  \
33218   [{'id': 'DRETSLABORAL', 'desc': 'Drets laboral...  3124607   
144914  [{'id': 'CCOO', 'desc': 'CCOO', 'main': False}...  3124607   

                                                    titol  \
33218   Els treballadors públics rebran a finals de no...   
144914  Els treb

In [5]:
clean_duplicates(id_df)

Conflicting rows for ID 3300462:
             id                                          entradeta  \
109085  3300462  Es reemeten les dues primeres temporades "Bola...   
146699  3300462  Es reemeten les dues primeres temporades "Bola...   

                                                      cos  redactor  \
109085  Els fans de la sèrie japonesa "Bola de Drac", ...  Redacció   
146699  Els fans de la sèrie japonesa "Bola de Drac", ...  Redacció   

                                                  imatges  
109085  [{'text': 'https://img.3cat.cat/multimedia/png...  
146699  [{'text': 'https://img.3cat.cat/multimedia/png...  


In [8]:
import ast  # To safely evaluate the string

# Now we will do the same with tags
# Initialize an empty list to hold flattened rows
tags_data = []

# Iterate over the DataFrame
for index, row in date_df.iterrows():
    tags_list = row['tags']
    
    # Check if it's a string and can be evaluated as a list
    if isinstance(tags_list, str):
        try:
            # Convert the string to a list using ast.literal_eval
            tags_list = ast.literal_eval(tags_list)
            # Now it's a list, so we can proceed
            if isinstance(tags_list, list):
                for item in tags_list:
                    tags_data.append(item)
        except (ValueError, SyntaxError) as e:
            print(f"Error parsing string at index {index}: {e}")
            
    elif isinstance(tags_list, list):  # This handles cases where it's already a list
        for item in tags_list:
            tags_data.append(item)
            
    else:
        # We have a NaN
        pass

# Create a new DataFrame from the flattened data
tags_df = pd.DataFrame(tags_data)

# Display the resulting DataFrame
print(tags_df)

                 id                desc   main
0            FRANCA              França   True
1           TRANSIT             Trànsit   True
2      JOCSOLIMPICS       Jocs Olímpics   True
3            MUSICA              Música  False
4              CUBA                Cuba   True
...             ...                 ...    ...
93872       CIENCIA             Ciència  False
93873       RECERCA  Recerca científica   True
93874           PSC                 PSC  False
93875          PSOE                PSOE   True
93876           ERC                 ERC   True

[93877 rows x 3 columns]


In [12]:
# Assuming df is your DataFrame
tags_final_df = tags_df.drop(columns=['main'])
tags_final_df = tags_final_df.drop_duplicates(keep='first')
tags_final_df = tags_final_df.reset_index(drop=True)

print(tags_final_df)

# print them in a txt file
tags_final_df.to_csv('tags.csv', index=False)

               id                         desc
0          FRANCA                       França
1         TRANSIT                      Trànsit
2    JOCSOLIMPICS                Jocs Olímpics
3          MUSICA                       Música
4            CUBA                         Cuba
..            ...                          ...
815  ECOIFINANCES          Economia i finances
816          CRIA                      Criança
817         ARRAN                        Arran
818   METEOCURIOS  Curiositats meteorològiques
819    ELFORASTER                  El foraster

[820 rows x 2 columns]


In [10]:
# We firstly group by id
id_groups = tags_final_df.groupby('id')

# Check if any 'id' has multiple 'desc' or 'main' values
ids_with_multiple_desc = id_groups['desc'].nunique()

# Step 2: Filter the ids with more than one unique 'desc' or 'main'
ids_with_multiple_desc = ids_with_multiple_desc[ids_with_multiple_desc > 1]

# Step 3: Output the results
print("IDs with multiple 'desc' values:")
print(ids_with_multiple_desc)

IDs with multiple 'desc' values:
id
ECOIFINANCES    2
ESTILDEVIDA     2
GASTRONOMIA     2
PODEM           2
TRUMP           2
VERIFICACIO     2
Name: desc, dtype: int64


In [11]:
# Assuming ids_with_multiple_desc is a Series with 'id' values as index
multiple_desc = tags_final_df[tags_final_df['id'].isin(ids_with_multiple_desc.index)]

# Display the result
print(multiple_desc)

               id                 desc
31          TRUMP               Trump 
111         PODEM         Unides Podem
130         TRUMP        Donald Trump 
182   GASTRONOMIA          Gastronomia
578   ESTILDEVIDA        Estil de vida
620   GASTRONOMIA         Gastronomia 
692   ESTILDEVIDA       Estils de vida
753   VERIFICACIO          Verificació
787         PODEM                Podem
811   VERIFICACIO    3CatInfo Verifica
812  ECOIFINANCES             Economia
815  ECOIFINANCES  Economia i finances


In [15]:
# Filter rows where 'tags' contains the string "EUAELECCIONS"
news_rows = date_df[date_df['tags'].apply(lambda x: isinstance(x, str) and "EUAELECCIONS" in x)]

print("Rows with 'EUAELECCIONS' in tags:")
print(news_rows)

Rows with 'EUAELECCIONS' in tags:
              data                                          entradeta  \
134186  2024-06-28  Biden vol donar una imatge més enèrgica despré...   
134243  2024-06-28  La mala imatge projectada pel president ha dis...   
134299  2024-06-28  Fredor i animadversió entre el president, que ...   
134339  2024-06-29  Time mostra el president a punt de caure de la...   
134672  2024-07-03  Segons el The New York Times, el president cre...   
...            ...                                                ...   
149304  17/11/2024  Sense cap experiència política, és defensor de...   
149833  22/11/2024  La designació arriba després que Matt Gaetz, l...   
149882  23/11/2024  Fundador d'un gestor de fons d'inversió, defen...   
149920  24/11/2024  En 20 dies, Cuba ha patit dos huracans, dos te...   
149997  25/11/2024  El fiscal també ha demanat desestimar l'acusac...   

                                               permatitle     avantitol  \
134186  Biden 

In [16]:
# we save the dataframe news_rows in a csv
news_rows.to_csv('eua_eleccions_dataset.csv', index=False)